### Similarity metrics of words in vector space model on CPU

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# download glove vectors from web
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove.6B.zip  

In [3]:
!head -1 glove.6B.50d.txt

the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581


In [4]:
def name_dtype_gen(dim_size=50):
    names = ['word']
    dtypes = ['str']
    for i in range(dim_size):
        names.append('dim_'+str(i+1))
        dtypes.append(np.float64)
    return names,dtypes

In [5]:
names, dtypes = name_dtype_gen(50)

In [6]:
%time df = pd.read_csv("glove.6B.50d.txt", delim_whitespace=True, names=names, quoting=3)  #ignore quoting 

CPU times: user 3.42 s, sys: 390 ms, total: 3.81 s
Wall time: 3.41 s


In [7]:
df.head()

,word,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_41,dim_42,dim_43,dim_44,dim_45,dim_46,dim_47,dim_48,dim_49,dim_50
0,the,0.418000,0.249680,-0.41242,0.12170,0.34527,-0.044457,-0.49688,-0.17862,-0.00066,...,-0.298710,-0.157490,-0.347580,-0.045637,-0.44251,0.187850,0.002785,-0.184110,-0.115140,-0.78581
1,",",0.013441,0.236820,-0.16899,0.40951,0.63812,0.477090,-0.42852,-0.55641,-0.36400,...,-0.080262,0.630030,0.321110,-0.467650,0.22786,0.360340,-0.378180,-0.566570,0.044691,0.30392
2,.,0.151640,0.301770,-0.16763,0.17684,0.31719,0.339730,-0.43478,-0.31086,-0.44999,...,-0.000064,0.068987,0.087939,-0.102850,-0.13931,0.223140,-0.080803,-0.356520,0.016413,0.10216
3,of,0.708530,0.570880,-0.47160,0.18048,0.54449,0.726030,0.18157,-0.52393,0.10381,...,-0.347270,0.284830,0.075693,-0.062178,-0.38988,0.229020,-0.216170,-0.225620,-0.093918,-0.80375
4,to,0.680470,-0.039263,0.30186,-0.17792,0.42962,0.032246,-0.41376,0.13228,-0.29847,...,-0.094375,0.018324,0.210480,-0.030880,-0.19722,0.082279,-0.094340,-0.073297,-0.064699,-0.26044


In [8]:
print(df.shape)

(400000, 51)


In [9]:
mappings = df['word']

In [10]:
mappings.shape

(400000,)

In [11]:
%time df = df.drop('word', axis=1)

CPU times: user 27.1 ms, sys: 36.3 ms, total: 63.3 ms
Wall time: 62.6 ms


In [12]:
%time mat = df[:400].to_numpy()

CPU times: user 701 µs, sys: 0 ns, total: 701 µs
Wall time: 575 µs


In [13]:
print(mat.shape)
print(mat.dtype)
mappings.head()

(400, 50)
float64


0    the
1      ,
2      .
3     of
4     to
Name: word, dtype: object

In [14]:
def ecludean_dist(a,b, dim_size):
    summ = 0
    for i in range(dim_size):
        summ += ((a[i]-b[i])**2)
    return math.sqrt(summ)

In [15]:
def dot(a, b, dim_size):
    summ = 0
    for i in range(dim_size):
        summ += (a[i]*b[i])
    return summ

In [16]:
def cosine_sim(a, b, dim_size):
    return dot(a,b, dim_size) / ( math.sqrt(dot(a, a, dim_size)) * math.sqrt(dot(b, b, dim_size)) )

In [17]:
def find_nearest(mat, out_1, out_2, dim_size, n):
    for idx in range(n):
        e = 9999999.0
        e_i = idx

        c = -1.0 
        c_i = idx

        for i in range(n):
            if i == idx:
                continue
            dist = ecludean_dist(mat[idx], mat[i], dim_size)
            csim = cosine_sim(mat[idx], mat[i], dim_size)
            if dist <= e:
                e_i = i
                e = dist
            if csim >= c:
                c_i = i
                c = csim

        out_1[idx] = e_i
        out_2[idx] = c_i

In [18]:
n = mat.shape[0]
dim_size = mat.shape[1]

In [19]:
out_1 = np.zeros(shape=n, dtype=np.int32)
out_2 = np.zeros(shape=n, dtype=np.int32)

In [20]:
%time find_nearest(mat, out_1, out_2, dim_size, n)

CPU times: user 17.1 s, sys: 3.13 ms, total: 17.1 s
Wall time: 17.1 s


For `n = 400`, cpu execution took 18 seconds.   
Since this algorithm is O(n^2), so if we increase the n by 10 times the execution time will be increased by 100 times.  
So, for n = `400000`, the execution time will be `18*(1000^2)` seconds = `~208 days` 